In this notebook we will perform classification on the Human activity recognition data
which you can find in this [link](https://www.kaggle.com/datasets/uciml/human-activity-recognition-with-smartphones). The database was built from the recordings of 30 study participants performing activities of daily living (ADL) while carrying a waist-mounted smartphone with embedded inertial sensors. The objective is to classify activities into one of the six activities performed. For this task you need to first download the data from aforementioned link and keep it in the `data` directory inside tutorials. Also, download the pandas library via `pip install pandas`

In [1]:
import pdb
import numpy as np
import pandas as pd 
import sys
sys.path.insert(0, '../')
from reservoirpy.nodes import Reservoir, Input, LinearRegression, RidgeRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from reservoirpy.datasets import mackey_glass
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [2]:
# First read the data using pandas and convert into numpy format
train = pd.read_csv("data/train.csv")
X_train= train.iloc[:,:-2].to_numpy()
Y_train= train.iloc[:,-1]
test= pd.read_csv("data/test.csv")
X_test=test.iloc[:,:-2].to_numpy()
Y_test= test.iloc[:,-1]

The labels i.e. Y_train and Y_test are in string format, thus, in order to convert them
into integers we will have to create a mapping between the string labels and their respective
class integers.

In [3]:
label2id = {"SITTING":0, "WALKING_DOWNSTAIRS":1,
"WALKING_UPSTAIRS":2, "STANDING":3, "WALKING":4,
"LAYING":5
}
Y_train = np.array([label2id[Y_train[i]] for i in range(len(Y_train))])
Y_test = np.array([label2id[Y_test[i]] for i in range(len(Y_test))])

Now, the target labels are in integer format, howvwer since we are performing regression, 
the output labels need to be in vectors. Thus we perform one-hot encoding on the training labels

In [4]:
print(f" Classes: {set(Y_train)} ")
Y_train_onehot = np.zeros((Y_train.shape[0], len(set(Y_train))))
for i in range(len(Y_train)):
	Y_train_onehot[i, Y_train[i]] = 1
Y_train = Y_train_onehot

 Classes: {0, 1, 2, 3, 4, 5} 


Now, we will create the model

In [5]:
source = Input()
reservoir = Reservoir(500, sr=0.9, lr=0.1)
readout = RidgeRegression()
model = source >> reservoir >> readout

We train the model by taking the last state from the reservoir node and storing it into lists.
Finally we call our fit method on the RidgeRegression readout node on the last satetes and target labels.

In [8]:
states_train = []
for x in X_train:
    states = reservoir.run(x, reset=True)
    states_train.append(states[-1, np.newaxis])
readout.fit(np.array(states_train).squeeze(), np.array(Y_train).squeeze())

Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 3472.11it/s]


Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 1199.40it/s]


Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 2949.58it/s]


Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 1440.85it/s]


Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 2666.44it/s]


...

Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 1166.70it/s]


Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 3013.15it/s]


Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 1212.23it/s]


Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 969.11it/s]


Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 3833.92it/s]
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Finally, we perform prediction on the test data and measure the accuracy.

In [10]:
states_test = []
for x in X_test:
    states = reservoir.run(x, reset=True)
    states_test.append(states[-1, np.newaxis])

Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 2898.62it/s]


Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 3086.32it/s]


Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 3622.02it/s]


Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 3440.77it/s]


Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 2584.29it/s]


...

Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 3125.41it/s]


Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 3830.41it/s]


Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 913.99it/s]


Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 3075.00it/s]


Running Reservoir-0: 100%|██████████| 1/1 [00:00<00:00, 3206.65it/s]


In [11]:
y_pred = readout.run(np.array(states_test).squeeze())
Y_pred_class = np.argmax(y_pred, axis=1)
score = accuracy_score(Y_test, Y_pred_class)
print("Accuracy: ", f"{score * 100:.3f} %")

Running RidgeRegression-0: 100%|██████████| 2947/2947 [00:00<00:00, 22309.89it/s]

Accuracy:  92.976 %
